In [1]:
!pip install xlsxwriter pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=f03b13ad4a50a8686b3c85dd876f06150f41e3755ce83d02466202c8ad437094
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive


In [2]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions - data conversion

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    """
    try:
        date_string = ' '.join(date_string.strip().split())

        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        if 'T' in date_string:
            return date_string.split('T')[0]

        # Format DD.MM.YYYY
        if re.match(r'\d{2}\.\d{2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
            changed_date = datetime.fromtimestamp(mktime(result))
            return format(changed_date.date())

        # Format DD-MM-YYYY
        if re.match(r'\d{2}-\d{2}-\d{4}', date_string):
            result = time.strptime(date_string, "%d-%m-%Y")
            changed_date = datetime.fromtimestamp(mktime(result))
            return format(changed_date.date())

        # Polski format z nazwami miesięcy
        lookup_table = {
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        return format(changed_date.date())
    except Exception as e:
        return "no date"


#%% functions - link extraction

def get_literature_links():
    """
    Pobiera wszystkie linki z sitemap-literature-1.xml do sitemap-literature-8.xml
    """
    base_url = "https://www.wywrota.pl/storage/sitemaps/"
    all_links = []

    print("="*60)
    print("KROK 1: Pobieranie linków z sitemap LITERATURE")
    print("="*60 + "\n")

    for i in range(1, 9):  # 1 do 8
        sitemap_url = f"{base_url}sitemap-literature-{i}.xml"

        try:
            print(f"Pobieranie sitemap-literature-{i}.xml...")
            r = requests.get(sitemap_url, timeout=10)

            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'xml')
                links = [loc.text.strip() for loc in soup.find_all('loc')]
                print(f"  ✓ Znaleziono {len(links)} linków")
                all_links.extend(links)
            else:
                print(f"  ✗ Status {r.status_code}")

            time.sleep(0.3)

        except Exception as e:
            print(f"  ✗ Błąd: {e}")

    # Deduplikacja
    all_links = list(set(all_links))

    print(f"\n{'='*60}")
    print(f"Łącznie znaleziono: {len(all_links)} unikalnych linków")
    print(f"{'='*60}\n")

    return all_links


#%% functions - scraping

def dictionary_of_article(article_link):
    """
    Pobiera szczegóły wiersza z literatura.wywrota.pl
    UWAGA: To są WIERSZE, nie artykuły blogowe!
    """
    try:
        r = requests.get(article_link, timeout=15)
        r.encoding = 'utf-8'

        if r.status_code != 200:
            errors.append(article_link)
            return

        html_text = r.text

        if 'error 404' in html_text.lower() or 'page not found' in html_text.lower():
            errors.append(article_link)
            return

        soup = BeautifulSoup(html_text, 'lxml')

        # Tytuł wiersza - w <h1 class="font-weight-bold mb-0">
        try:
            title_element = soup.find('h1', class_='font-weight-bold')
            if not title_element:
                title_element = soup.find('h1')

            if title_element:
                title = title_element.get_text(strip=True)
            else:
                title = "no title"
        except:
            title = "no title"

        # Autor wiersza - w <h2 class="mb-3">
        try:
            author_element = soup.find('h2', class_='mb-3')
            if author_element:
                author = author_element.get_text(strip=True)
            else:
                # Fallback - breadcrumb
                breadcrumb = soup.find('ol', class_='breadcrumb')
                if breadcrumb:
                    items = breadcrumb.find_all('li', class_='breadcrumb-item')
                    if len(items) >= 2:
                        author_link = items[1].find('a')
                        if author_link:
                            author = author_link.get_text(strip=True)
                        else:
                            author = "no author"
                    else:
                        author = "no author"
                else:
                    author = "no author"
        except:
            author = "no author"

        # Data publikacji - w <div class="text-muted txt-sm"> po "·"
        try:
            date_div = soup.find('div', class_='text-muted txt-sm')
            if date_div:
                date_text = date_div.get_text()
                # Format: "Wiersz · 22 września 1999"
                if '·' in date_text:
                    parts = date_text.split('·')
                    if len(parts) >= 2:
                        date_text = parts[1].strip()
                        date_of_publication = date_change_format(date_text)
                    else:
                        date_of_publication = "no date"
                else:
                    date_of_publication = date_change_format(date_text)
            else:
                date_of_publication = "no date"
        except:
            date_of_publication = "no date"

        # Tekst wiersza - w <div class="article-content small-line-breaks mb-3">
        try:
            poem_content = soup.find('div', class_='article-content')
            if poem_content:
                # Zachowaj strukturę z <br> jako nowe linie
                text = poem_content.get_text(separator='\n', strip=False)
                # Usuń nadmiarowe puste linie
                text = re.sub(r'\n{3,}', '\n\n', text)
                text = text.strip()
            else:
                text = "no text"
        except:
            text = "no text"

        # Kategoria - z breadcrumb lub z txt-sm (np. "Wiersz")
        try:
            # Pobierz typ z txt-sm
            date_div = soup.find('div', class_='text-muted txt-sm')
            if date_div:
                text = date_div.get_text()
                if '·' in text:
                    category_type = text.split('·')[0].strip()
                else:
                    category_type = "Literatura"
            else:
                category_type = "Literatura"

            category = category_type
        except:
            category = "Literatura"

        # Tagi
        try:
            tag_links = soup.find_all('a', rel='tag')
            if tag_links:
                tags = [tag.get_text(strip=True) for tag in tag_links]
                tags_str = ' | '.join(tags)
            else:
                tags_str = None
        except:
            tags_str = None

        # Linki zewnętrzne - raczej nie będzie w wierszach
        external_links = None

        # Zdjęcia - może być ilustracja
        try:
            images = []

            # Szukamy obrazków w contencie
            for img in soup.find_all('img'):
                img_url = img.get('src') or img.get('data-src')
                if img_url:
                    # Pomijamy małe ikony i avatary
                    if 'icon' not in img_url.lower() and 'avatar' not in img_url.lower():
                        if not img_url.startswith('http'):
                            img_url = 'https://literatura.wywrota.pl' + img_url
                        if img_url not in images:
                            images.append(img_url)

            has_images = len(images) > 0
            photos_links = ' | '.join(images) if images else None
        except:
            has_images = False
            photos_links = None

        # Filmy - raczej nie będzie w wierszach
        has_videos = False

        result = {
            "Link": article_link,
            "Data publikacji": date_of_publication,
            "Tytuł artykułu": title.replace('\xa0', ' '),
            "Tekst artykułu": text,
            "Autor": author,
            "Kategoria": category,
            "Tagi": tags_str,
            "Linki zewnętrzne": external_links,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(result)

    except Exception as e:
        errors.append(article_link)


#%% main execution

if __name__ == "__main__":
    print("\n" + "="*60)
    print("SCRAPER LITERATURA.WYWROTA.PL - WIERSZE")
    print("="*60 + "\n")

    # KROK 1: Pobierz linki
    article_links = get_literature_links()

    if not article_links:
        print("Nie znaleziono linków!")
        exit(1)

    # KROK 2: Scrapuj wiersze
    all_results = []
    errors = []

    print("="*60)
    print("KROK 2: Scraping wierszy")
    print("="*60 + "\n")

    max_workers = 10
    print(f"Używam {max_workers} równoległych wątków\n")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(dictionary_of_article, article_links), total=len(article_links)))

    # KROK 3: Zapisz wyniki
    timestamp = datetime.today().date()

    print(f"\n{'='*60}")
    print("KROK 3: Zapisywanie wyników")
    print("="*60)

    # JSON
    json_file = f'literatura_wywrota_{timestamp}.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)
    print(f"  ✓ {json_file}")

    # Excel
    excel_file = f"literatura_wywrota_{timestamp}.xlsx"
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(excel_file,
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)
    print(f"  ✓ {excel_file}")

    # RAPORT KOŃCOWY
    print(f"\n{'='*60}")
    print("RAPORT KOŃCOWY")
    print("="*60)
    print(f"Pobranych wierszy: {len(all_results)}")
    print(f"Błędów: {len(errors)}")

    if errors and len(errors) <= 10:
        print(f"\nLinki z błędami:")
        for error_link in errors:
            print(f"  - {error_link}")
    elif errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        print(f"  ... i {len(errors) - 10} więcej")

    print(f"\n{'='*60}")
    print("GOTOWE!")
    print("="*60 + "\n")


SCRAPER LITERATURA.WYWROTA.PL - WIERSZE

KROK 1: Pobieranie linków z sitemap LITERATURE

Pobieranie sitemap-literature-1.xml...
  ✓ Znaleziono 5000 linków
Pobieranie sitemap-literature-2.xml...
  ✓ Znaleziono 5000 linków
Pobieranie sitemap-literature-3.xml...
  ✓ Znaleziono 5000 linków
Pobieranie sitemap-literature-4.xml...
  ✓ Znaleziono 5000 linków
Pobieranie sitemap-literature-5.xml...
  ✓ Znaleziono 5000 linków
Pobieranie sitemap-literature-6.xml...
  ✓ Znaleziono 5000 linków
Pobieranie sitemap-literature-7.xml...
  ✓ Znaleziono 5000 linków
Pobieranie sitemap-literature-8.xml...
  ✓ Znaleziono 3196 linków

Łącznie znaleziono: 38196 unikalnych linków

KROK 2: Scraping wierszy

Używam 10 równoległych wątków



100%|██████████| 38196/38196 [56:14<00:00, 11.32it/s]
/tmp/ipython-input-1870437219.py:325: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Posts', index=False)



KROK 3: Zapisywanie wyników
  ✓ literatura_wywrota_2026-01-13.json
  ✓ literatura_wywrota_2026-01-13.xlsx

RAPORT KOŃCOWY
Pobranych wierszy: 1710
Błędów: 36486

Linki z błędami (pierwsze 10):
  - https://literatura.wywrota.pl/wiersz/58470-yaro-na-portalu-co-raz-wiecej.html
  - https://literatura.wywrota.pl/opowiadanie/63826-marek-jastrzab--niegdysiejsze-sniegi.html
  - https://literatura.wywrota.pl/opowiadanie/11178-endi-shiroishi-obcy.html
  - https://literatura.wywrota.pl/wiersz-klasyka/40298-tadeusz-micinski-kallypso.html
  - https://literatura.wywrota.pl/wiersz/35507-ja-i-inni-pytanie-do-boga.html
  - https://literatura.wywrota.pl/wiersz/21286-dziza-upojenie.html
  - https://literatura.wywrota.pl/wiersz-klasyka/41877-zbigniew-herbert-longobardowie.html
  - https://literatura.wywrota.pl/wiersz/977-oryphiel-lesniczowka.html
  - https://literatura.wywrota.pl/wiersz/14222-elkam-list.html
  - https://literatura.wywrota.pl/wiersz-klasyka/42658-jorge-luis-borges-aleksandria-641-ad.html
 

In [3]:
df.head()

,Link,Data publikacji,Tytuł artykułu,Tekst artykułu,Autor,Kategoria,Tagi,Linki zewnętrzne,Zdjęcia/Grafika,Filmy,Linki do zdjęć
0,https://literatura.wywrota.pl/wiersz/59216-mar...,2019-10-09,BLIŻEJ,\n\n Wiersz\n ...,Marcin Strugalski,Wiersz,None,None,True,False,https://literatura.wywrota.pl/assets/img/wywro...
1,https://literatura.wywrota.pl/wiersz-klasyka/4...,no date,Arachne,Arachne była córką kupca farb w jednym z miast...,Zbigniew Herbert,Literatura,None,None,True,False,https://literatura.wywrota.pl/assets/img/wywro...
2,https://literatura.wywrota.pl/wiersz/59639-yar...,2020-01-13,Tylko Ty,\n\n Wiersz\n ...,Yaro,Wiersz,None,None,True,False,https://literatura.wywrota.pl/assets/img/wywro...
3,https://literatura.wywrota.pl/wiersz-klasyka/2...,no date,Matka,O zmierzchu przy oknie\nMatka trąca nogą biegu...,Staff Leopold,Literatura,None,None,True,False,https://literatura.wywrota.pl/assets/img/wywro...
4,https://literatura.wywrota.pl/wiersz/32417-voy...,2012-01-30,(haiq CCXIX),\n\n Wiersz\n ...,Voyteq Hieronymus de Borkovsky,Wiersz,None,None,True,False,https://literatura.wywrota.pl/assets/img/wywro...
